# Using Xarray-Spatial and Datashader to Quantify Pharmacy Deserts

## Environment Set-up

1. `conda create -n pharmacy-deserts python=3.8`

2. `conda activate pharmacy-deserts`

3. `conda install -c conda-forge jupyterlab`

4. `pip3 install numba==0.48.0`

5. `conda install datashader`

6. `conda install geopandas`

7. `conda install -c conda-forge xarray-spatial`

8. `conda install -c pyviz spatialpandas`

9. `conda install -c conda-forge geos=3.7.1`



In [ ]:
import datashader as ds
import numpy as np
import pandas as pd
import geopandas as gpd

from datashader.transfer_functions import stack
from datashader.transfer_functions import shade
from datashader.transfer_functions import set_background
from datashader.colors import inferno

from xrspatial.classify import natural_breaks
from xrspatial.classify import binary
from xrspatial import proximity

from spatialpandas import GeoDataFrame

## Load data

### Data Sources
- [Facilities CSV](https://rxopen.org/)
- [US Block Groups Shapefile](https://hub.arcgis.com/datasets/esri::usa-block-groups/data?geometry=-166.940%2C28.846%2C167.571%2C67.170)
- [US Counties Shapefile](https://hub.arcgis.com/datasets/48f9af87daa241c4b267c5931ad3b226_0?geometry=-166.940%2C28.846%2C167.571%2C67.170)

In [ ]:
# Load Pharmacies and add out x, y fields based on CalcLocation
pharmacy_df = pd.read_csv(r"./data/facilities.csv")
coords = pharmacy_df['CalcLocation'].str.split(',', expand=True)
pharmacy_df['y'] = np.array(coords[0], dtype='float64')
pharmacy_df['x'] = np.array(coords[1], dtype='float64')
pharmacy_df

In [ ]:
# Load Census Block Groups and Calculate Percent over 65 years-old
blockgroup_df = GeoDataFrame(gpd.read_file(r"./data/USA_Block_Groups.shp"))
blockgroup_df['ABOVE_65'] = blockgroup_df[['AGE_65_74', 'AGE_75_84', 'AGE_85_UP']].sum(axis=1) 
blockgroup_df['PCT_ABOVE_65'] = blockgroup_df['ABOVE_65'] / blockgroup_df['POP2010']
blockgroup_df

In [ ]:
# Load Census County and Calculate Percent over 65 years-old 
county_df = GeoDataFrame(gpd.read_file(r"./data/USA_Counties.shp"))
county_df['ZONE_ID'] = county_df['OBJECTID'].astype(np.int16)
county_df

## Define Study Area

In [ ]:
x_range = (-124.848974, -66.885444)
y_range = (24.396308, 49.384358)

W = 1600
H = 800

cvs = ds.Canvas(plot_width=W, plot_height=H,
                x_range=x_range, y_range=y_range)

county_mask = cvs.polygons(county_df, geometry='geometry')
set_background(shade(county_mask, cmap='#333333', alpha=255), 'black')

### Create a "Distance to Nearest Pharmacy" Layer & Classify into 5 Groups

In [ ]:
pharmacy_raster = cvs.points(pharmacy_df, 'x', 'y')
proximity_raster = proximity(pharmacy_raster, distance_metric='GREAT_CIRCLE').where(county_mask)
proximity_raster.data[~np.isfinite(proximity_raster.data)] = 0.0

proximity_classifed = natural_breaks(proximity_raster, k=5).where(county_mask)

image_pharmacy = shade(proximity_classifed, cmap=inferno, alpha=255)
image_pharmacy = set_background(image_pharmacy, 'black')
image_pharmacy

### Create an Age Layer  & Classify into 5 Groups

In [ ]:
age_raster = cvs.polygons(blockgroup_df, geometry='geometry', agg=ds.mean('PCT_ABOVE_65'))
age_raster.data[~np.isfinite(age_raster.data)] = 0.0
age_classifed = natural_breaks(age_raster, k=5).where(county_mask)

age_image = shade(age_classifed, cmap=inferno, alpha=255)
age_image = set_background(age_image, 'black')
age_image

### Combine layers to highlight seniors at risk from pharmacy deserts

In [ ]:
pharmacy_deserts = binary(proximity_classifed, [4.0])
older_regions = binary(age_classifed, [4.0])
target_deserts = (pharmacy_deserts * older_regions).where(county_mask)
target_deserts_img = shade(target_deserts, cmap=['#333333', 'fuchsia'], alpha=255, how='linear')
set_background(target_deserts_img, 'black')

###  Summarize seniors at risk by county

In [ ]:
from datashader.colors import Set1

counties_raster = cvs.polygons(county_df, geometry='geometry', agg=ds.max('OBJECTID'))
counties_image = shade(counties_raster, cmap=Set1, alpha=225, how='linear')
set_background(counties_image, 'black')

## Zonal Statistics

Zonal statistics allows for calculating summary statistics for specific areas or zones within a datashader aggregate. Zones are defined by creating an integer aggregate where the cell values are zone_ids. The output of zonal statistics is a Pandas dataframe containing summary statistics for each zone based on a value raster.


In [ ]:
from xrspatial import zonal_stats

# summary functions
zonal_funcs = dict(pharmacy_desert_mean=lambda z: z.mean())

# zones
counties_raster.data = counties_raster.data.astype(np.int64)

# values to summarize
target_deserts.data = target_deserts.data.astype(np.int8)

# execute summary functions on each zone and take top 10
results = zonal_stats(counties_raster, target_deserts, zonal_funcs)

### Join result back to counties layer

In [ ]:

cols = ['pharmacy_desert_mean', 'NAME', 'STATE_FIPS', 'geometry']
final_df = pd.merge(county_df, results, left_on='ZONE_ID', right_index=True)[cols]
final_df.nlargest(10, 'pharmacy_desert_mean')

In [ ]:
from xrspatial import hillshade

counties_raster = cvs.polygons(county_df, geometry='geometry', agg=ds.max('OBJECTID'))

desert_raster = cvs.polygons(final_df.nlargest(10, 'pharmacy_desert_mean'),
                             geometry='geometry',
                             agg=ds.mean('pharmacy_desert_mean'))

county_mask = cvs.polygons(county_df, geometry='geometry')

In [ ]:
img = stack(
    shade(county_mask, cmap=['#333333'], alpha=255),
    shade(counties_raster, cmap=['#333333', '#ffffff'], alpha=25),
    shade(desert_raster, cmap=['#333333','#ff0000'], alpha=200),
    shade(hillshade(desert_raster), cmap=['#333333', '#ff0000'], alpha=100),
)
set_background(img, 'black')